In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
from imutils import paths
import os
import pandas as pd
from pytesseract import pytesseract

# pytesseract.pytesseract.tesseract_cmd = "C:\Program Files\Tesseract-OCR\\tesseract.exe"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# !apt install tesseract-ocr

In [4]:
imagePaths = list(paths.list_images("/content/drive/MyDrive/datad"))



In [5]:
def prepro_angle(img):
    filtered_image= cv2.bilateralFilter(img, 5, 50, 50)
    ret,threshold_img=cv2.threshold(filtered_image,140,255,cv2.THRESH_BINARY)
    element=np.ones((2,2),np.uint8)
    threshold_img=cv2.erode(threshold_img,element)
    return threshold_img

def prepro_space(img):
    filtered_image= cv2.bilateralFilter(img, 5, 50, 50)
    ret,threshold_img=cv2.threshold(filtered_image,135,255,cv2.THRESH_BINARY_INV)
    return threshold_img

def prepro_pressure(img):
    inverted=255-img
    bilateral = cv2.bilateralFilter(inverted, 5, 50, 50)
    ret,thresh=cv2.threshold(bilateral,110,255,cv2.THRESH_TOZERO)
    return thresh



In [6]:
def line_spacing(threshold_img):
    letter_size=average_letter_size=var_letter_size=0
    TOP_MARGIN=LINE_SPACING=WORD_SPACING=0.0
    lines=list()
    def horizontal_projection ()->list:
        (h,w)=np.shape(threshold_img)
        print(h,w)
        sum_rows=list()
        for j in range (0,h-1):
            row=threshold_img[j:j+1,0:w]
            sum_rows.append(np.sum(row))
        return sum_rows
    hplist=horizontal_projection()

    topMargincount=0
    for sum in hplist:
        if(sum<=255):
            topMargincount=topMargincount+1
        else:
            break
    lineTop=0
    lineBottom=0
    spaceTop=0
    SpaceBottom=0
    setLineTop=True
    setSpaceTop=True
    includeNextSpace=True
    space_zero=list()
    for i in range(0,len(hplist)):
        if hplist[i]<3000:
            if setSpaceTop:
                spaceTop=i
                setSpaceTop=False
            SpaceBottom=i+1
            if i<len(hplist)-1:
                if(hplist[i+1]<3000):
                    continue
            if includeNextSpace:
                space_zero.append(SpaceBottom-spaceTop)
            else:
                previous=0
                if(len(space_zero)==0):
                    previous=0
                else:
                    previous=space_zero.pop()
                space_zero.append(previous+SpaceBottom-lineTop)
            setSpaceTop=True
        if hplist[i]>3000:
            if setLineTop:
                lineTop=i
                setLineTop=False
            lineBottom=i+1
            if i<len(hplist)-1:
                if hplist[i+1]>3000:
                    continue
                if(lineBottom-lineTop<20):
                    includeNextSpace=False
                    setLineTop=True
                    continue
            includeNextSpace=True
            lines.append([lineTop,lineBottom])
            setLineTop=True
    threshold=15000
    new_space_row_count=0
    total_row_count=0
    total_line_count=0
    flag=False
    for i,line in enumerate(lines):
        segment=hplist[line[0]:line[1]]
        for j,sum in enumerate(segment):
            if sum<threshold:
                new_space_row_count=new_space_row_count+1
            else:
                total_row_count=total_row_count+1
                flag=True
        if flag:
            total_line_count=total_line_count+1
            flag=False
    total_space_row_count=new_space_row_count+np.sum(space_zero[1:-1])
    #average_line_spacing=float(total_space_row_count)/total_line_count
    var_line_spacing=new_space_row_count
    if(len(space_zero)!=0 and len(space_zero)<=2):

       var_line_spacing=var_line_spacing+space_zero[-1];
    elif(len(space_zero)>2):
        var_line_spacing=var_line_spacing+ np.var(space_zero[1:-1])

    return var_line_spacing






In [7]:
def pen_pressure(img):
    element=np.ones((3,3),np.uint8)
    skel=np.zeros(np.shape(img),np.uint8)
    original_variance=np.var(img)
    while True:
        open=cv2.erode(img,element)
        temp=img-open
        eroded=cv2.erode(img,element)
        skel=np.bitwise_or(skel,temp)
        img=eroded
    # print('a')
        if(np.all(img==0)):
            break

    thinned=np.var(skel)
    pres=original_variance-thinned
    return pres


In [8]:
def slant_angle(threshold_img):
    def getVerticalProjectionProfile(image):

    # Convert black spots to ones
    #image[image == 0]   = 1
    # Convert white spots to zeros
        image[image == 255] = 1

        vertical_projection = np.sum(image, axis = 0)

        return vertical_projection
    s_function=np.zeros(9,np.float64)
    count_=np.zeros(9,np.uint64)
    data=pytesseract.image_to_data(threshold_img,output_type='dict')
    boxes = len(data['level'])
    words=[]
    for i in range(boxes ):
        words.append((data['left'][i], data['top'][i], data['width'][i], data['height'][i]))
    angles=[ -45,-30, -15, -5, 0, 5, 15, 30,45]
    for i in range (0,9):
        angle=angles[i]
        s_temp=0.0
        count=0
        for word in words:
            (x,y,w,h)=word
        #   if(x<0 or y<0):
        #      continue
            if(h<20 or w<20):
                continue
        #   if(w<0):
        #      print(w)
            original=threshold_img[x:x+h,y:y+w]
            shift=(np.tan(angle*np.pi/180)*h)/2
            pad_length=abs(int(shift))
        #   if(pad_length<0):
        #    print(pad_length)
            x=int(h)
            y=int(w+pad_length*2)
            if(x<=0 or y<=0):
                continue
            blank_image=np.zeros((int(x),int(y),3),np.uint8)
            new_image=cv2.cvtColor(blank_image,cv2.COLOR_BGR2GRAY)
        #   if(y<w):
        #      print(y,w)
            if(w!=np.shape(original)[1] or h!=np.shape(original)[0]):
                continue
            new_image[:,pad_length:w+pad_length]=original
            (h,w)=np.shape(new_image)[:2]
            x1=w/2
            y1=0
            x2=w/4
            y2=h
            x3=3*w/4
            y3=h
            pts1=np.float32([[x1,y1],[x2,y2],[x3,y3]])
            pts2=np.float32([[x1+shift,y1],[x2-shift,y2],[x3-shift,y3]])
            M = cv2.getAffineTransform(pts1, pts2)
            deslanted=cv2.warpAffine(new_image,M,(w,h))
            #print(deslanted)
            vp=getVerticalProjectionProfile(deslanted.copy())
            #print(vp)
            for vpk in vp:
                if(vpk==0):
                 continue
                num_fgpixel=vpk
            # print(num_fgpixel,h)
                if num_fgpixel<h/3:
                 continue

                s_temp=s_temp+num_fgpixel/h
                count=count+1
        if(count==0):
         continue

        s_function[i]=s_function[i]+s_temp/count
        count_[i]=count_[i]+1
    index_max=np.argmax(s_function)
    SLANT_ANGLE=angles[index_max]
    return SLANT_ANGLE


In [9]:
def baseline_angle(dilated_img):
    contours, hierarchy = cv2.findContours(dilated_img,
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    angle_list=list()
    for ctr in contours:
            center,dim,angle=cv2.minAreaRect(ctr)
            if(dim[1]<20): #as paper asked to ignore boxes with height less than 20 pixels
             continue
            if angle>80:
                angle=90-angle
            elif angle<-45:
                angle=angle+90
            angle_list.append(angle)
    baseline_angle=np.var(angle_list)
    return baseline_angle


In [12]:
cols=['Pen Pressure', 'Line Spacing','Slant Angle', 'Baseline angle','Category','Link']
features=pd.DataFrame(columns=cols )
for image in imagePaths:

    lable = os.path.split(image)[1].split('_')[-1].split('.')[0]
    val=bool(0)
    if(lable=="yes"):
        val=bool(1)

    img=cv2.imread(image)

    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=cv2.resize(img,(900,900))
    img_ang=prepro_angle(img)
    lst=[pen_pressure(prepro_pressure(img)),line_spacing(prepro_space(img)),slant_angle(img_ang),baseline_angle(img_ang),val,image]
    features.loc[len(features.index)]=lst


900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900
900 900


In [13]:
features.to_csv('features.csv',header=True,index=False)

In [ ]:
pd.isna(features).sum()